In [ ]:
"""
Created on Mon Mar 22 10:42 2021

This script is to prepare the thermal forcing term for the linear regression for simple params

author : Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.melt_functions as meltf
from basal_melt_param.constants import *
import basal_melt_param.T_S_profile_functions as tspf
from scipy import stats

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
nemo_run = 'isfru94'

In [ ]:

if nemo_run in ['ctrl94','isf94','isfru94']:
    inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/NEMO_'+nemo_run+'_ANT_STEREO/'
    inputpath_mask = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    outputpath_simple = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
    outputpath_plumes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/PLUMES/nemo_5km_'+nemo_run+'/'
    outputpath = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
else:
    inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
    inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    outputpath_simple = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
    outputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
    outputpath = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/'

plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:

if nemo_run in ['ctrl94','isf94','isfru94']:
    file_isf_orig = xr.open_mfdataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc')
else:
    file_isf_orig = xr.open_mfdataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_Nisf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_Nisf)

In [ ]:
param_var_of_int = file_isf[['ISF_mask', 'latitude', 'longitude', 'front_bot_depth_max']]
#plume_charac = xr.open_dataset(outputpath_plumes+'nemo_5km_plume_characteristics.nc')
plume_charac = xr.open_dataset(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')

In [ ]:
ice_draft_pos = file_other_cut['corrected_isfdraft']
ice_draft_neg = -ice_draft_pos
isf_conc = file_conc_cut['isfdraft_conc']

In [ ]:
bottom = False
HUB = False

if bottom:
    thermal_forcing_1map = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_bottom_for_tuning_corrected_oneFRIS.nc')
    salinity_forcing_1map = xr.open_dataset(outputpath_simple+'input_salinity_2D_bottom_for_tuning_corrected_oneFRIS.nc')
elif HUB:
    thermal_forcing_1map = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_HUB_for_tuning_corrected_oneFRIS.nc')
    salinity_forcing_1map = xr.open_dataset(outputpath_simple+'input_salinity_2D_HUB_for_tuning_corrected_oneFRIS.nc')
else:
    thermal_forcing_1map = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_for_tuning_corrected_oneFRIS.nc')
    salinity_forcing_1map = xr.open_dataset(outputpath_simple+'input_salinity_2D_for_tuning_corrected_oneFRIS.nc')

In [ ]:
nonweighted_thermal_forcing = thermal_forcing_1map['thermal_forcing_2D']
nonweighted_salinity = salinity_forcing_1map['salinity_depth_of_int']

weighted_thermal_forcing = thermal_forcing_1map['thermal_forcing_2D']*isf_conc
weighted_salinity = salinity_forcing_1map['salinity_depth_of_int']*isf_conc

thermal_forcing_all = []

if bottom:
    simple_param_types = ['linear_local_bottom', 'quadratic_local_bottom', 'quadratic_local_locslope_bottom', 'quadratic_local_cavslope_bottom', 'quadratic_mixed_mean_bottom', 'quadratic_mixed_locslope_bottom','quadratic_mixed_cavslope_bottom']
else:
    simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_local_locslope', 'quadratic_local_cavslope', 'quadratic_mixed_mean', 'quadratic_mixed_locslope','quadratic_mixed_cavslope']

for kisf in tqdm(file_isf['Nisf']):
    isf_mask = file_isf['ISF_mask'] == kisf
    
    nonweighted_thermal_forcing_nisf = nonweighted_thermal_forcing.where(isf_mask, drop=True)
    nonweighted_salinity_nisf = nonweighted_salinity.where(isf_mask, drop=True)

    weighted_thermal_forcing_nisf = weighted_thermal_forcing.where(isf_mask, drop=True)
    weighted_avg_thermal_forcing = (weighted_thermal_forcing_nisf.sum(['x','y'])/isf_conc.where(isf_mask, drop=True).sum(['x','y']))#.compute()

    weighted_salinity_nisf = weighted_salinity.where(isf_mask, drop=True)
    weighted_avg_salinity = (weighted_salinity_nisf.sum(['x','y'])/isf_conc.where(isf_mask, drop=True).sum(['x','y']))#.compute()
    
    isf_conc_nisf = isf_conc.where(isf_mask, drop=True)

    angles_nisf_simple = np.sin(plume_charac['alpha'].sel(option='simple').where(isf_mask, drop=True))
    angles_nisf_locslope = np.sin(plume_charac['alpha'].sel(option='appenB').where(isf_mask, drop=True))
    
    thermal_forcing_kisf = None

    for mparam in simple_param_types:
        
        if bottom:
            mparam0 = mparam[:-7:]
        else:
            mparam0 = mparam

        if mparam0 == 'linear_local':
            thermal_forcing_term = weighted_thermal_forcing_nisf.sum(['x','y'])
        elif mparam == 'quadratic_local':
            thermal_forcing_term = (nonweighted_thermal_forcing_nisf
                                    * nonweighted_salinity_nisf
                                    * abs(nonweighted_thermal_forcing_nisf) * isf_conc_nisf).sum(['x','y'])
        elif mparam0 == 'quadratic_local_locslope':
            thermal_forcing_term = (nonweighted_thermal_forcing_nisf
                                    * nonweighted_salinity_nisf
                                    * abs(nonweighted_thermal_forcing_nisf) * angles_nisf_locslope * isf_conc_nisf).sum(dim=['x','y']) 
        elif mparam0 == 'quadratic_local_cavslope':
            thermal_forcing_term = (nonweighted_thermal_forcing_nisf 
                                    * nonweighted_salinity_nisf
                                    * abs(nonweighted_thermal_forcing_nisf) * angles_nisf_simple * isf_conc_nisf).sum(dim=['x','y']) 
        elif mparam0 == 'quadratic_mixed_mean':
            thermal_forcing_term = abs(weighted_avg_thermal_forcing) * weighted_avg_salinity * (nonweighted_thermal_forcing_nisf * isf_conc_nisf).sum(['x','y'])
        elif mparam0 == 'quadratic_mixed_locslope':
            thermal_forcing_term = abs(weighted_avg_thermal_forcing) * weighted_avg_salinity * (nonweighted_thermal_forcing_nisf 
                                                                        * angles_nisf_locslope * isf_conc_nisf).sum(dim=['x','y']) 
        elif mparam == 'quadratic_mixed_cavslope':
            thermal_forcing_term = abs(weighted_avg_thermal_forcing) * weighted_avg_salinity * (nonweighted_thermal_forcing_nisf 
                                                                        * angles_nisf_simple * isf_conc_nisf).sum(dim=['x','y']) 

        if 'option' in thermal_forcing_term.coords:
            thermal_forcing_term = thermal_forcing_term.drop('option')
        
        thermal_forcing_kisf = meltf.merge_over_dim(thermal_forcing_term, thermal_forcing_kisf, 'param', mparam)
    

    thermal_forcing_all.append(thermal_forcing_kisf)
    
thermal_forcing_merged = xr.concat(thermal_forcing_all, dim='Nisf')
        

In [ ]:
#thermal_forcing_merged.to_dataset(name='thermal_forcing_term').to_netcdf(outputpath_simple+'thermal_forcing_term_for_linreg.nc','w') 
#thermal_forcing_merged.to_dataset(name='thermal_forcing_term').to_netcdf(outputpath_simple+'thermal_forcing_term_for_linreg_contshelf_only.nc','w') 
#thermal_forcing_merged.to_dataset(name='thermal_forcing_term').to_netcdf(outputpath_simple+'thermal_forcing_term_for_linreg_minus_0-5.nc','w') 
#thermal_forcing_merged.to_dataset(name='thermal_forcing_term').to_netcdf(outputpath_simple+'thermal_forcing_term_for_linreg_corrected.nc','w') 
if bottom:
    #thermal_forcing_merged.to_dataset(name='thermal_forcing_term').to_netcdf(outputpath_simple+'thermal_forcing_term_bottom_for_linreg_corrected.nc','w') 
    thermal_forcing_merged.to_dataset(name='thermal_forcing_term').to_netcdf(outputpath_simple+'thermal_forcing_term_bottom_for_linreg_corrected_oneFRIS.nc','w') 
elif HUB:
    thermal_forcing_merged.to_dataset(name='thermal_forcing_term').to_netcdf(outputpath_simple+'thermal_forcing_term_HUB_for_linreg_corrected_oneFRIS.nc','w') 
else:
    #thermal_forcing_merged.to_dataset(name='thermal_forcing_term').to_netcdf(outputpath_simple+'thermal_forcing_term_for_linreg_corrected.nc','w') 
    thermal_forcing_merged.to_dataset(name='thermal_forcing_term').to_netcdf(outputpath_simple+'thermal_forcing_term_for_linreg_corrected_oneFRIS.nc','w') 



In [ ]:
alpha_all = plume_charac['alpha'].sel(option='appenB').where(salinity_forcing_1map['salinity_depth_of_int'].isel(profile_domain=0,time=0)>0) 

In [ ]:
thermal_forcing_merged.param

In [ ]:
np.sin(uf.weighted_mean(alpha_all, ['x','y'], isf_conc))

In [ ]:
nisf_list = file_isf.Nisf.values.aslist()